##DATA SCRAPPING AND IMPORTING

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.1 MB/s eta 0:00:00


In [ ]:
import json
import random
from faker import Faker

# Initialize Faker
fake = Faker()

# Define the number of rows
num_rows = 200000

# Define the structure of the JSON data
data = []

# Generate the data
for i in range(1, num_rows + 1):
    row = {
        "ID": i,
        "Store_id": random.randint(1, 1000),
        "Store_Type": random.choice(["Supermarket", "Grocery", "Hypermarket", "Convenience"]),
        "Location_Type": random.choice(["Urban", "Suburban", "Rural"]),
        "Region_Code": fake.state_abbr(),
        "Date": fake.date_this_decade().strftime("%Y-%m-%d"),
        "Holiday": random.choice([True, False]),
        "Discount": random.choice([True, False]),
        "#Order": random.randint(1, 1000),
        "Sales": round(random.uniform(10, 1000), 2)
    }
    data.append(row)

# Save the data to a JSON file
with open("store_data.json", "w") as json_file:
    json.dump(data, json_file, indent=4)

print(f"{num_rows} rows of data have been generated and saved to 'store_data.json'")


200000 rows of data have been generated and saved to 'store_data.json'


In [ ]:
import pandas as pd

# Sample JSON data
json_data = pd.read_json("/content/store_data.json")

# Convert JSON to DataFrame
df = pd.DataFrame(json_data)

# Save DataFrame to Excel file
excel_file_path = "output.xlsx"
df.to_excel(excel_file_path, index=False)

print(f"JSON data has been converted to {excel_file_path}")


JSON data has been converted to output.xlsx


##DATA PREPROCESSING

In [ ]:
import pandas as pd
import numpy as np


df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/DATASET/output.xlsx")

In [ ]:
df.head(2)

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,#Order,Sales
0,1,709,Grocery,Suburban,NM,2022-10-31,True,True,661,750.92
1,2,988,Convenience,Rural,MP,2023-10-05,False,True,201,854.04


In [ ]:
df.shape

(200000, 10)

In [ ]:
df.isnull().sum()

,0
ID,0
Store_id,0
Store_Type,0
Location_Type,0
Region_Code,0
Date,0
Holiday,0
Discount,0
#Order,0
Sales,0


**{'Supermarket': 1, 'Grocery': 2, 'Hypermarket': 3, 'Convenience': 4}**

In [ ]:
df['Store_Type'] = df['Store_Type'].map({'Supermarket': 1, 'Grocery': 2, 'Hypermarket': 3, 'Convenience': 4})

In [ ]:
df['Location_Type'] = df['Location_Type'].map({'Urban': 1, 'Suburban': 2, 'Rural': 3})
df['Holiday'] = df['Holiday'].map({True: 1, False: 0})

In [ ]:
df.head(2)

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,#Order,Sales
0,1,709,2,2,NM,2022-10-31,1,True,661,750.92
1,2,988,4,3,MP,2023-10-05,0,True,201,854.04


In [ ]:
x = pd.DataFrame(df[['Store_Type','Location_Type','Holiday','Discount']])
y = pd.DataFrame(df['#Order'])

In [ ]:
x.head(2)

,Store_Type,Location_Type,Holiday,Discount
0,2,2,1,True
1,4,3,0,True


##ML ALGO

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=42,test_size=0.2)

In [ ]:
import lightgbm as lgb
model = lgb.LGBMRegressor()
model.fit(x_train,y_train)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014819 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13
[LightGBM] [Info] Number of data points in the train set: 160000, number of used features: 4
[LightGBM] [Info] Start training from score 501.193425


LGBMRegressor()

In [ ]:
y_prediction = model.predict(x_test)
data = pd.DataFrame(data = {"predicted orders":y_prediction.flatten()})
print(data.head(2))

   predicted orders
0        496.723905
1        508.435793


In [ ]:
data

,predicted orders
0,496.723905
1,508.435793
2,498.375799
3,494.301466
4,492.033273
...,...
39995,500.973706
39996,498.073408
39997,498.673737
39998,494.301466
